### Step 1. Download the college normalized dataset:
#### gsutil cp gs://cs327e-open-access/college_normalized.zip .
#### unzip college_normalized.zip

### Step 2: Run code samples in sequence to populate Firestore database

In [1]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

In [3]:
batch = db.batch()

df = pd.read_csv('/home/jupyter/college_normalized/class.csv', sep=',', header=1, lineterminator='\n')
rows = df.values.tolist()

for row in rows:
    
    record = {}
    record['cno'] = row[0]
    record['cname'] = row[1]
    record['credits'] = row[2]
    
    class_ref = db.collection('class').document(row[0])
    
    batch.set(class_ref, record)
    
batch.commit()

[update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 },
 update_time {
   seconds: 1614976973
   nanos: 28770000
 }]

In [4]:
batch = db.batch()

df = pd.read_csv('/home/jupyter/college_normalized/student.csv', sep=',', header=0, lineterminator='\n')
student_rows = df.values.tolist()

df = pd.read_csv('/home/jupyter/college_normalized/takes.csv', sep=',', header=0, lineterminator='\n')
takes_rows = df.values.tolist()

for student_row in student_rows:
    
    student_record = {}
    student_record['sid'] = student_row[0]
    student_record['fname'] = student_row[1]
    student_record['lname'] = student_row[2]
    student_record['dob'] = student_row[3]
    student_record['status'] = student_row[4]
    
    student_ref = db.collection('student').document(student_row[0])
    
    batch.set(student_ref, student_record)
    
    for takes_row in takes_rows:
        if student_row[0] == takes_row[0]:
            
            class_record = {}
            class_record['cno'] = takes_row[1]
            class_record['grade'] = takes_row[2]
            
            classes_ref = student_ref.collection('classes').document(takes_row[1])
        
            batch.set(classes_ref, class_record)
    
    
    batch.commit()
    

In [8]:
batch = db.batch()

df = pd.read_csv('/home/jupyter/college_normalized/instructor.csv', sep=',', header=0, lineterminator='\n')
instructor_rows = df.values.tolist()

df = pd.read_csv('/home/jupyter/college_normalized/teaches.csv', sep=',', header=0, lineterminator='\n')
teaches_rows = df.values.tolist()

for instructor_row in instructor_rows:
    
    instructor_record = {}
    instructor_record['tid'] = instructor_row[0]
    instructor_record['instructor_name'] = instructor_row[1]
    instructor_record['dept'] = instructor_row[2]
    
    instructor_ref = db.collection('instructor').document(instructor_row[0])
    
    batch.set(instructor_ref, instructor_record)
    
    for teaches_row in teaches_rows:
        
        if instructor_row[0] == teaches_row[0]:
            
            class_record = {}
            class_record['cno'] = teaches_row[1]
            class_record['cname'] = teaches_row[2]
            class_record['credits'] = teaches_row[3]
            
            classes_ref = instructor_ref.collection('classes').document(teaches_row[1])
        
            batch.set(classes_ref, class_record)
    
    batch.commit()

### Step 3: Run some queries

In [9]:
student_ref = db.collection('student').document('paulg')
result = student_ref.get()

if result.exists:
    print(f'{result.id} => {result.to_dict()}')
else:
    print('No such student')

paulg => {'fname': 'Paul', 'status': 'CUR', 'lname': 'Gore', 'sid': 'paulg', 'dob': '2000-09-17'}


In [10]:
student_ref = db.collection('student')
query = student_ref.where('status', '==', 'CUR')
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

aprilz => {'fname': 'April', 'dob': '2000-10-01', 'sid': 'aprilz', 'lname': 'Lopez', 'status': 'CUR'}
bzen26 => {'dob': '1998-04-22', 'fname': 'Biswa', 'sid': 'bzen26', 'lname': 'Zen', 'status': 'CUR'}
jc => {'fname': 'James', 'dob': '2000-04-22', 'status': 'CUR', 'sid': 'jc', 'lname': 'Cowe'}
jerryh => {'lname': 'Hargrove', 'dob': '1999-01-03', 'fname': 'Jerry', 'status': 'CUR', 'sid': 'jerryh'}
paulg => {'sid': 'paulg', 'dob': '2000-09-17', 'status': 'CUR', 'fname': 'Paul', 'lname': 'Gore'}
sudeepa4 => {'dob': '2001-10-01', 'sid': 'sudeepa4', 'lname': 'Roy', 'status': 'CUR', 'fname': 'Sudeepa'}


In [13]:
query = db.collection('instructor').document('mitra').collection('classes')

results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

CS313E => {'cno': 'CS313E', 'credits': 3, 'cname': 'Elements of Software Engineering'}
CS329E => {'credits': 3, 'cno': 'CS329E', 'cname': 'Elements of Web Programming'}


In [16]:
classes = db.collection_group('classes').where('credits', '==', 3)
docs = classes.stream()
for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

CS303E => {'cno': 'CS303E', 'credits': 3, 'cname': 'Elements of Computers and Programming'}
CS326E => {'cno': 'CS326E', 'credits': 3, 'cname': 'Elements of Networking'}
CS347 => {'cno': 'CS347', 'cname': 'Data Management', 'credits': 3}
CS373 => {'credits': 3, 'cno': 'CS373', 'cname': 'Software Engineering'}
CS331E => {'cno': 'CS331E', 'credits': 3, 'cname': 'Elements of Software Engineering II'}
M328K => {'cno': 'M328K', 'credits': 3, 'cname': 'Intro to Number Theory'}
CS313E => {'cname': 'Elements of Software Engineering', 'credits': 3, 'cno': 'CS313E'}
CS329E => {'cname': 'Elements of Web Programming', 'cno': 'CS329E', 'credits': 3}
M362K => {'cname': 'Probability I', 'credits': 3, 'cno': 'M362K'}
M362K => {'cname': 'Probability I', 'cno': 'M362K', 'credits': 3}
CS327E => {'cno': 'CS327E', 'cname': 'Elements of Databases', 'credits': 3}
M358K => {'credits': 3, 'cno': 'M358K', 'cname': 'Applied Statistics'}


In [18]:
instructor_ref = db.collection('instructor')
results = instructor_ref.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')
    sresults = instructor_ref.document(result.id).collection('classes').stream()
    
    for sresult in sresults:
        print(f'{sresult.id} => {sresult.to_dict()}')

bulko => {'tid': 'bulko', 'instructor_name': 'Bill Bulko', 'dept': 'Computer Science'}
CS303E => {'cno': 'CS303E', 'credits': 3, 'cname': 'Elements of Computers and Programming'}
cannata => {'tid': 'cannata', 'instructor_name': 'Phil Cannata', 'dept': 'Computer Science'}
CS326E => {'credits': 3, 'cname': 'Elements of Networking', 'cno': 'CS326E'}
CS347 => {'cname': 'Data Management', 'credits': 3, 'cno': 'CS347'}
CS347  => {'cno': 'CS347 '}
downing => {'tid': 'downing', 'instructor_name': 'Glenn Downing', 'dept': 'Computer Science'}
CS373 => {'credits': 3, 'cname': 'Software Engineering', 'cno': 'CS373'}
CS373  => {'cno': 'CS373 '}
fares => {'dept': 'Computer Science', 'tid': 'fares', 'instructor_name': 'Fares Fraij'}
CS331E => {'credits': 3, 'cno': 'CS331E', 'cname': 'Elements of Software Engineering II'}
koch => {'instructor_name': 'Hans Koch', 'dept': 'Mathematics', 'tid': 'koch'}
M328K => {'cno': 'M328K', 'cname': 'Intro to Number Theory', 'credits': 3}
M328K  => {'cno': 'M328K '}
